# ESG Scores calculation

In this file I'll be calculating the ESG scores for the firms. The methodology is similar to the one applied by Refinitiv, however I exclude variables that are relative to the firm being, at the given period, carbon-intensive

In [1]:
import pandas as pd
import numpy as np

In [2]:
_2016 = pd.read_excel("./ESG_Calculations.xlsx", sheet_name = "2016")
_2017 = pd.read_excel("./ESG_Calculations.xlsx", sheet_name = "2017")
_2018 = pd.read_excel("./ESG_Calculations.xlsx", sheet_name = "2018")
_2018_2 = pd.read_excel("./ESG_Calculations.xlsx", sheet_name = "2018_2")
rules = pd.read_excel("./ESG_Calculations.xlsx", sheet_name = "Rules")

In [3]:
_2016.head()

,Firm,Emission Reduction Processes/Policy Emissions Reduction,Emission Reduction Objectives/Targets Emissions Reduction,Biodiversity Impact Reduction,NOx and SOx Emissions Reduction Initiatives,e-Waste Reduction Initiatives,Emissions Trading,Environmental Partnerships,ISO 14000 or EMS Certified Percent,Environmental Restoration Initiatives,...,Resource Efficiency Objectives/Targets Water Efficiency,Resource Efficiency Objectives/Targets Energy Efficiency,Materials Sourcing Environmental Criteria,Toxic Substances Reduction Initiatives,Green Buildings,Water Recycled,Environmental Supply Chain Selection Management,Environmental Supply Chain Partnership Termination,Environmental Supply Chain Monitoring,Value - Resource Reduction/Renewable Energy Use
0,544294,N,N,N,N,N,N,N,NaN,N,...,N,N,N,N,N,NaN,N,N,NaN,NaN
1,923587,N,N,N,N,N,N,N,NaN,N,...,N,N,N,N,N,NaN,N,N,NaN,NaN
2,938972,N,N,N,N,N,N,N,NaN,N,...,N,N,N,N,N,NaN,N,N,Y,NaN
3,916328,Y,Y,N,N,Y,N,Y,NaN,N,...,Y,Y,Y,N,Y,NaN,Y,N,Y,0.00052
4,87851X,Y,Y,N,N,N,N,Y,NaN,N,...,Y,Y,Y,Y,N,NaN,Y,N,Y,NaN


In [4]:
rules.head()

,DS_Code,Title,Units,Polarity,Syntax
0,ENERDP0051,Emission Reduction Processes/Policy Emissions ...,Y/N,Positive,Emission Reduction
1,ENERDP0161,Emission Reduction Objectives/Targets Emission...,Y/N,Positive,Emission Reduction
2,ENERDP019,Biodiversity Impact Reduction,Y/N,Positive,Emission Reduction
3,ENERDP033,NOx and SOx Emissions Reduction Initiatives,Y/N,Positive,Emission Reduction
4,ENERDP063,e-Waste Reduction Initiatives,Y/N,Positive,Emission Reduction


In [5]:
#Now let's create one dataframe with the scores

scores = pd.DataFrame(columns = _2016.columns, index = _2016.index)
scores["Firm"] = _2016["Firm"]

In [6]:
scores.head()

,Firm,Emission Reduction Processes/Policy Emissions Reduction,Emission Reduction Objectives/Targets Emissions Reduction,Biodiversity Impact Reduction,NOx and SOx Emissions Reduction Initiatives,e-Waste Reduction Initiatives,Emissions Trading,Environmental Partnerships,ISO 14000 or EMS Certified Percent,Environmental Restoration Initiatives,...,Resource Efficiency Objectives/Targets Water Efficiency,Resource Efficiency Objectives/Targets Energy Efficiency,Materials Sourcing Environmental Criteria,Toxic Substances Reduction Initiatives,Green Buildings,Water Recycled,Environmental Supply Chain Selection Management,Environmental Supply Chain Partnership Termination,Environmental Supply Chain Monitoring,Value - Resource Reduction/Renewable Energy Use
0,544294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,923587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,938972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,916328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,87851X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
dummies = rules.loc[rules["Units"] == "Y/N"]
values_positive = rules.loc[rules["Polarity"] == "Positive"].loc[rules["Units"]!="Y/N"]
values_negative = rules.loc[rules["Polarity"] == "Negative"].loc[rules["Units"]!="Y/N"]

In [8]:
#Scores for dummies: Easiest ones to compute

for score in dummies["Title"]:
    #Score for Yes
    Y = (len(_2016[score].loc[_2016[score]== "N"]) + 0.5*len(_2016[score].loc[_2016[score]== "Y"]))/(len(_2016[score]))
    #Score for No
    N = 0.5*len(_2016[score].loc[_2016[score]=="N"])/len(_2016[score])
    NA = 0
    
    scores[score] = np.where(_2016[score] == "Y", Y, N) #inserting values
    
    #replacing with zero in case there are Nans in the series
    if len(_2016.loc[_2016[score].isna()]) > 0:
        scores[score] = np.where(_2016[score].isna(), 0, scores[score])

In [9]:
#Scores positive for values :

for score in values_positive["Title"]:
    score_list = list()
    _2016.set_index("Firm", inplace = True, drop = True) #Setting index to firms
    
    for firm in _2016.index:
        score_list.append((len(_2016[score].dropna().loc[_2016[score] < _2016.loc[firm,score]]) + 0.5*len(_2016[score].dropna().loc[_2016[score] < _2016.loc[firm,score]]))/len(_2016[score]))
         
    #Removing index
    _2016.reset_index("Firm", inplace = True)
    #insert list in DataFrame scores:
    scores[score] = score_list
    

In [10]:
scores.loc[:,values_positive["Title"]].describe()

,ISO 14000 or EMS Certified Percent,Value - Emission Reduction/Waste Recycling Ratio,Value - Product Innovation/Environmental R&D Expenditures,Value - Product Innovation/Renewable Energy Supply,Water Recycled,Value - Resource Reduction/Renewable Energy Use
count,2102.000000,2102.000000,2102.000000,2102.000000,2102.000000,2102.000000
mean,0.001544,0.007428,0.000005,0.000022,0.000773,0.001205
std,0.007326,0.026126,0.000115,0.001027,0.004924,0.006839
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.041389,0.149857,0.003568,0.047098,0.047812,0.059943


In [11]:

values_negative

,DS_Code,Title,Units,Polarity,Syntax
11,ENERDP103,Self-Reported Environmental Fines,Number (Currency = Local Reporting Currency)/NA,Negative,Emission Reduction


In [12]:
#Score for the only negative value: "Self-Reported Environmental Fines"

score = "Self-Reported Environmental Fines"
score_list = list()
_2016.set_index("Firm", inplace = True, drop = True) #Setting index to firms

for firm in _2016.index:
    score_list.append(-(len(_2016[score].dropna().loc[_2016[score] < _2016.loc[firm,score]]) + 0.5*len(_2016[score].dropna().loc[_2016[score] < _2016.loc[firm,score]]))/len(_2016[score]))

#Removing index
_2016.reset_index("Firm", inplace = True)
#insert list in DataFrame scores:
scores[score] = score_list

In [13]:
scores[score].describe()

count    2102.000000
mean       -0.001819
std         0.011068
min        -0.087060
25%        -0.000000
50%        -0.000000
75%        -0.000000
max        -0.000000
Name: Self-Reported Environmental Fines, dtype: float64

# Repeat The Process, for 2017: Paris 

In [14]:
scores17 = pd.DataFrame(columns = _2017.columns, index = _2017.index)
scores17["Firm"] = _2017["Firm"]

#Scores for dummies: Easiest ones to be fair

for score in dummies["Title"]:
    #Score for Yes
    Y = (len(_2017[score].loc[_2017[score]== "N"]) + 0.5*len(_2017[score].loc[_2017[score]== "Y"]))/(len(_2017[score]))
    #Score for No
    N = 0.5*len(_2017[score].loc[_2017[score]=="N"])/len(_2017[score])
    NA = 0
    
    scores17[score] = np.where(_2017[score] == "Y", Y, N) #inserting values
    
    #replacing with zero in case there are Nans in the series
    if len(_2017.loc[_2017[score].isna()]) > 0:
        scores17[score] = np.where(_2017[score].isna(), 0, scores17[score])
        
#Scores positive for values :

for score in values_positive["Title"]:
    score_list = list()
    _2017.set_index("Firm", inplace = True, drop = True) #Setting index to firms
    
    for firm in _2017.index:
        score_list.append((len(_2017[score].dropna().loc[_2017[score] < _2017.loc[firm,score]]) + 0.5*len(_2017[score].dropna().loc[_2017[score] < _2017.loc[firm,score]]))/len(_2017[score]))
         
    #Removing index
    _2017.reset_index("Firm", inplace = True)
    #insert list in DataFrame scores:
    scores17[score] = score_list

# Repeat The Process, for 2018: To be used on Long-Term Analysis

In [15]:
scores18 = pd.DataFrame(columns = _2018.columns, index = _2018.index)
scores18["Firm"] = _2018["Firm"]

#Scores for dummies: Easiest ones to be fair

for score in dummies["Title"]:
    #Score for Yes
    Y = (len(_2018[score].loc[_2018[score]== "N"]) + 0.5*len(_2018[score].loc[_2018[score]== "Y"]))/(len(_2018[score]))
    #Score for No
    N = 0.5*len(_2018[score].loc[_2018[score]=="N"])/len(_2018[score])
    NA = 0
    
    scores18[score] = np.where(_2018[score] == "Y", Y, N) #inserting values
    
    #replacing with zero in case there are Nans in the series
    if len(_2018.loc[_2018[score].isna()]) > 0:
        scores18[score] = np.where(_2018[score].isna(), 0, scores18[score])
        
#Scores positive for values :

for score in values_positive["Title"]:
    score_list = list()
    _2018.set_index("Firm", inplace = True, drop = True) #Setting index to firms
    
    for firm in _2018.index:
        score_list.append((len(_2018[score].dropna().loc[_2018[score] < _2018.loc[firm,score]]) + 0.5*len(_2018[score].dropna().loc[_2018[score] < _2018.loc[firm,score]]))/len(_2018[score]))
         
    #Removing index
    _2018.reset_index("Firm", inplace = True)
    #insert list in DataFrame scores:
    scores18[score] = score_list

# Repeat The Process, for 2018_2: To be used on Long-Term Analysis

In [16]:
scores18_2 = pd.DataFrame(columns = _2018_2.columns, index = _2018_2.index)
scores18_2["Firm"] = _2018_2["Firm"]

#Scores for dummies: Easiest ones to be fair

for score in dummies["Title"]:
    #Score for Yes
    Y = (len(_2018_2[score].loc[_2018_2[score]== "N"]) + 0.5*len(_2018_2[score].loc[_2018_2[score]== "Y"]))/(len(_2018_2[score]))
    #Score for No
    N = 0.5*len(_2018_2[score].loc[_2018_2[score]=="N"])/len(_2018_2[score])
    NA = 0
    
    scores18_2[score] = np.where(_2018_2[score] == "Y", Y, N) #inserting values
    
    #replacing with zero in case there are Nans in the series
    if len(_2018_2.loc[_2018_2[score].isna()]) > 0:
        scores18_2[score] = np.where(_2018_2[score].isna(), 0, scores18_2[score])
        
#Scores positive for values :

for score in values_positive["Title"]:
    score_list = list()
    _2018_2.set_index("Firm", inplace = True, drop = True) #Setting index to firms
    
    for firm in _2018_2.index:
        score_list.append((len(_2018_2[score].dropna().loc[_2018_2[score] < _2018_2.loc[firm,score]]) + 0.5*len(_2018_2[score].dropna().loc[_2018_2[score] < _2018_2.loc[firm,score]]))/len(_2018_2[score]))
         
    #Removing index
    _2018_2.reset_index("Firm", inplace = True)
    #insert list in DataFrame scores:
    scores18_2[score] = score_list

# Computing final Env_Scores

Now that we have the scores per variable, let's compute the full environmental score:
Each individual category score gets same weight

Env_Score1 --> score considering all the variables

Env_Score1d --> score considering just the dummies

Env_Score2 --> Not computed in this document, but is the original score 

And finally all these scores for Paris:

Env_Score1P;  Env_Score1dP;  EnvScore2P

In [17]:
#Env_Score1:
scores["Env_Score1"] = scores.iloc[:,1:].sum(axis=1)/(len(scores.columns)-1)

#Env_Score1d:
scores["Env_Score1d"] = scores.loc[:,dummies["Title"]].sum(axis=1)/(len(dummies)-1)

#Env_Score1P:
scores17["Env_Score1P"] = scores17.iloc[:,1:].sum(axis=1)/(len(scores.columns)-1)

#Env_ScoredP:
scores17["Env_Score1dP"] = scores17.loc[:,dummies["Title"]].sum(axis=1)/(len(dummies)-1)

#Env_Score1P:
scores18["Env_Score1_18"] = scores18.iloc[:,1:].sum(axis=1)/(len(scores.columns)-1)

#Env_ScoredP:
scores18["Env_Score1d_18"] = scores18.loc[:,dummies["Title"]].sum(axis=1)/(len(dummies)-1)

#Env_Score1P:
scores18_2["Env_Score1_18"] = scores18_2.iloc[:,1:].sum(axis=1)/(len(scores.columns)-1)

#Env_ScoredP:
scores18_2["Env_Score1d_18"] = scores18_2.loc[:,dummies["Title"]].sum(axis=1)/(len(dummies)-1)

In [18]:
scores.describe()

,Emission Reduction Processes/Policy Emissions Reduction,Emission Reduction Objectives/Targets Emissions Reduction,Biodiversity Impact Reduction,NOx and SOx Emissions Reduction Initiatives,e-Waste Reduction Initiatives,Emissions Trading,Environmental Partnerships,ISO 14000 or EMS Certified Percent,Environmental Restoration Initiatives,Staff Transport Impact Reduction Initiatives,...,Materials Sourcing Environmental Criteria,Toxic Substances Reduction Initiatives,Green Buildings,Water Recycled,Environmental Supply Chain Selection Management,Environmental Supply Chain Partnership Termination,Environmental Supply Chain Monitoring,Value - Resource Reduction/Renewable Energy Use,Env_Score1,Env_Score1d
count,2102.000000,2102.000000,2102.000000,2102.000000,2102.000000,2102.000000,2102.000000,2102.000000,2102.000000,2102.000000,...,2102.000000,2102.000000,2102.000000,2102.000000,2102.000000,2102.00000,2102.000000,2102.000000,2102.000000,2102.000000
mean,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.001544,0.500000,0.500000,...,0.500000,0.500000,0.500000,0.000773,0.500000,0.50000,0.011229,0.001205,0.382953,0.500432
std,0.224371,0.178040,0.129925,0.091568,0.168385,0.087217,0.203631,0.007326,0.139591,0.161894,...,0.181716,0.130315,0.199438,0.004924,0.218646,0.15612,0.026800,0.006839,0.085674,0.111434
min,0.360371,0.425547,0.463606,0.482636,0.434824,0.484301,0.395100,0.000000,0.457422,0.440533,...,0.421741,0.463368,0.400809,0.000000,0.371313,0.44529,0.000000,0.000000,0.320325,0.418887
25%,0.360371,0.425547,0.463606,0.482636,0.434824,0.484301,0.395100,0.000000,0.457422,0.440533,...,0.421741,0.463368,0.400809,0.000000,0.371313,0.44529,0.000000,0.000000,0.320325,0.418887
50%,0.360371,0.425547,0.463606,0.482636,0.434824,0.484301,0.395100,0.000000,0.457422,0.440533,...,0.421741,0.463368,0.400809,0.000000,0.371313,0.44529,0.000000,0.000000,0.335031,0.438118
75%,0.860371,0.425547,0.463606,0.482636,0.434824,0.484301,0.395100,0.000000,0.457422,0.440533,...,0.421741,0.463368,0.400809,0.000000,0.871313,0.44529,0.000000,0.000000,0.423266,0.553502
max,0.860371,0.925547,0.963606,0.982636,0.934824,0.984301,0.895100,0.041389,0.957422,0.940533,...,0.921741,0.963368,0.900809,0.047812,0.871313,0.94529,0.075167,0.059943,0.697613,0.902547


In [19]:
#Exporting Final Scores:
final = scores.loc[:,["Firm","Env_Score1","Env_Score1d"]]
finalp = scores17.loc[:,["Firm","Env_Score1P","Env_Score1dP"]]
final18 = scores18.loc[:,["Firm","Env_Score1_18","Env_Score1d_18"]]
final18_2 = scores18_2.loc[:,["Firm","Env_Score1_18","Env_Score1d_18"]]

In [20]:
""""
with pd.ExcelWriter("./Env_Scores.xlsx") as writer:
    final.to_excel(writer, sheet_name='2016')
    finalp.to_excel(writer, sheet_name='2017')
    final18.to_excel(writer, sheet_name='2018')
    final18_2.to_excel(writer, sheet_name='2018_2')